# Image Classification
> - 참고: 책 <<밑바닥부터 시작하는 딥러닝>>, pp.96-106 
> - MNIST dataset: https://github.com/youbeebee/deeplearning_from_scratch/tree/master/dataset 
> - 가중치: https://github.com/youbeebee/deeplearning_from_scratch/blob/master/ch3.%EC%8B%A0%EA%B2%BD%EB%A7%9D/sample_weight.pkl

In [3]:
import sys, os

# # 부모 디렉토리의 파일을 가져올 수 있도록 설정
# sys.path.append(os.pardir)
from datasets.mnist import load_mnist

## 1. 데이터셋 확인

In [6]:
# normalize: 픽셀 값을 0.0~1.0사이 값으로 정규화할 것인가?
# flatten: 입력 이미지를 1차원 배열로 만들 것인가?
# one_hot_label: 레이블을 one-hot encoding 형태로 저장할 것인가?
(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False)

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


## 2. 신경망 구현
- 입력층 뉴런 784개(이미지 크기), 출력층 뉴런 10개(0~9까지의 숫자)
- 은닉층은 두 개. 첫 번째 은닉층에는 50개의 뉴런, 두 번째 은닉층에는 100개의 뉴런

In [9]:
from datasets.mnist import load_mnist
import pickle
import numpy as np

In [18]:
def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

def softmax(x):
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

In [19]:
def get_data():
    (x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=True)
    return x_test, y_test

def init_network():
    # 가중치 매개변수 가져오기
    with open('assets/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)

    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [21]:
x, t = get_data()
network = init_network()

print(f'X의 shape: {x.shape}')
print(f'y의 shape: {t.shape}')
print('network의 key를 출력합니다.')
print(network.keys())

X의 shape: (10000, 784)
y의 shape: (10000,)
network의 key를 출력합니다.
dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])


In [22]:
accuracy_count = 0
for i in range(len(x)):
    y = predict(network, x[i])

    # 출력 중 값이 가장 높은(확률이 가장 높은) 원소 인덱스 얻기
    p = np.argmax(y)
    
    if p == t[i]:
        accuracy_count += 1

print(f'정확도: {str(float(accuracy_count / len(x)))}')

정확도: 0.9352


## 3. 배치 처리

In [23]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_count = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i: i+batch_size]
    y_batch = predict(network, x_batch)
    
    p = np.argmax(y_batch, axis=1)
    accuracy_count += np.sum(p == t[i:i+batch_size])

print(f'정확도: {str(float(accuracy_count / len(x)))}')

정확도: 0.9352
